In [1]:
import pandas as pd
import numpy as np
import re, string
import nltk
import pickle
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

df = pd.read_csv('/Users/shanecooke/Desktop/Official_GitLab/CompleteData.csv')
df

,Platform,Comment,Hateful
0,Reddit,Damn I thought they had strict gun laws in Ger...,0
1,Reddit,I dont care about what it stands for or anythi...,0
2,Reddit,It's not a group it's an idea lol,0
3,Reddit,So it's not just America!,0
4,Reddit,The dog is a spectacular dancer considering he...,0
...,...,...,...
2995,4Chan,This is what Kike shilling looks like. Ryan ha...,1
2996,4Chan,"Not bait, they're right",0
2997,4Chan,I like this one a lot.,0
2998,4Chan,Kikes making money off heroin what’s new,1


In [2]:
def preProcessText(text):
    text = text.lower() 
    text = text.strip()  
    text = re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

def stopwordRemoval(string):
    stop = [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(stop)

wl = WordNetLemmatizer()

def tagMapping(tag):
    if tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def lemmatization(string):
    words = nltk.pos_tag(word_tokenize(string))
    temp = [wl.lemmatize(tag[0], tagMapping(tag[1])) for idx, tag in enumerate(words)]
    return " ".join(temp)

def finalCleaning(string):
    return lemmatization(stopwordRemoval(preProcessText(string)))

df['clean_text'] = df['Comment'].apply(lambda x: finalCleaning(x))
df

,Platform,Comment,Hateful,clean_text
0,Reddit,Damn I thought they had strict gun laws in Ger...,0,damn think strict gun law germany
1,Reddit,I dont care about what it stands for or anythi...,0,dont care stand anything connect like shield
2,Reddit,It's not a group it's an idea lol,0,group idea lol
3,Reddit,So it's not just America!,0,america
4,Reddit,The dog is a spectacular dancer considering he...,0,dog spectacular dancer consider two leave foot
...,...,...,...,...
2995,4Chan,This is what Kike shilling looks like. Ryan ha...,1,kike shill look like ryan do redpill people fa...
2996,4Chan,"Not bait, they're right",0,bait right
2997,4Chan,I like this one a lot.,0,like one lot
2998,4Chan,Kikes making money off heroin what’s new,1,kike make money heroin whats new


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import texthero as hero
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import HashingVectorizer
from gensim.models import Word2Vec
import tensorflow_hub as hub

df['tfidf'] = (hero.tfidf(df['clean_text'], max_features=1000))

df['tfidf1'] = (hero.tfidf(df['clean_text'], max_features=2000))

df['tfidf2'] = (hero.tfidf(df['clean_text'], max_features=3000))

card_docs = [TaggedDocument(doc.split(' '), [i]) for i, doc in enumerate(df.clean_text)]
model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 10)
model.build_vocab(card_docs)
model.train(card_docs, total_examples=model.corpus_count, epochs=model.epochs)
temp = [model.infer_vector((df['clean_text'][i].split(' '))) for i in range(0,len(df['clean_text']))]
dtv = np.array(temp).tolist()
df['Doc2Vec'] = dtv

card_docs = [TaggedDocument(doc.split(' '), [i]) for i, doc in enumerate(df.clean_text)]
model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 20)
model.build_vocab(card_docs)
model.train(card_docs, total_examples=model.corpus_count, epochs=model.epochs)
temp = [model.infer_vector((df['clean_text'][i].split(' '))) for i in range(0,len(df['clean_text']))]
dtv = np.array(temp).tolist()
df['Doc2Vec1'] = dtv

card_docs = [TaggedDocument(doc.split(' '), [i]) for i, doc in enumerate(df.clean_text)]
model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 40)
model.build_vocab(card_docs)
model.train(card_docs, total_examples=model.corpus_count, epochs=model.epochs)
temp = [model.infer_vector((df['clean_text'][i].split(' '))) for i in range(0,len(df['clean_text']))]
dtv = np.array(temp).tolist()
df['Doc2Vec2'] = dtv

vectorizer = HashingVectorizer(n_features=100)
hashed = vectorizer.transform(df['clean_text']).toarray()
hashList = np.array(hashed).tolist()
df['hashing'] = hashList

vectorizer = HashingVectorizer(n_features=300)
hashed = vectorizer.transform(df['clean_text']).toarray()
hashList = np.array(hashed).tolist()
df['hashing1'] = hashList

vectorizer = HashingVectorizer(n_features=500)
hashed = vectorizer.transform(df['clean_text']).toarray()
hashList = np.array(hashed).tolist()
df['hashing2'] = hashList

df

,Platform,Comment,Hateful,clean_text,tfidf,tfidf1,tfidf2,Doc2Vec,Doc2Vec1,Doc2Vec2,hashing,hashing1,hashing2
0,Reddit,Damn I thought they had strict gun laws in Ger...,0,damn think strict gun law germany,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0032718100119382143, 0.004408253822475672, ...","[-0.08356256037950516, 0.17799748480319977, 0....","[-0.08634454011917114, 0.18881037831306458, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, -0.4082482904638631, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Reddit,I dont care about what it stands for or anythi...,0,dont care stand anything connect like shield,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0018799236277118325, 0.005465839523822069, ...","[-0.048040103167295456, 0.10511787235736847, 0...","[0.023344872519373894, 0.02822037972509861, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Reddit,It's not a group it's an idea lol,0,group idea lol,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.00534618878737092, 0.008697368204593658, 0....","[-0.02423957549035549, 0.0676441639661789, 0.0...","[-0.03381773456931114, 0.10322832316160202, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Reddit,So it's not just America!,0,america,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.004059546627104282, -0.0004826425283681601...","[-0.02149725891649723, 0.03351963311433792, 0....","[-0.06831774860620499, 0.12614895403385162, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Reddit,The dog is a spectacular dancer considering he...,0,dog spectacular dancer consider two leave foot,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0009879578137770295, 0.010099951177835464,...","[-0.08533000200986862, 0.17648042738437653, 0....","[-0.11566188931465149, 0.2142035961151123, 0.1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,4Chan,This is what Kike shilling looks like. Ryan ha...,1,kike shill look like ryan do redpill people fa...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.1984729755786248e-05, -0.000203559786314144...","[-0.06838290393352509, 0.13812002539634705, 0....","[-0.0753791481256485, 0.2228599190711975, 0.11...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2996,4Chan,"Not bait, they're right",0,bait right,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0048239585012197495, -0.0038504567928612232...","[-0.022948168218135834, 0.050892189145088196, ...","[-0.0495951883494854, 0.10468236356973648, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score
from statistics import mean
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

print("TFIDF:\n")
params = ["Max_features = 1000", "Max_features = 2000", "Max_features = 3000"]

def wordEmbeddingOptimisationTFIDF():
    for i in range(0, 3):
        X, y = df[df.columns[i+4]].tolist(), df.Hateful
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        
        classifier = RandomForestClassifier(n_estimators=1000)
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        
        prec = precision_score(y_test, y_pred)
        rec = recall_score(y_test, y_pred)
        f1score = f1_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(params[i])
        print("-------------------")
        print("Accuracy: ", round(accuracy_score(y_test, y_pred), 6))
        print("Precision: ", round(precision_score(y_test, y_pred), 2))
        print("Recall: ", round(recall_score(y_test, y_pred),2))
        print("F1 Score: ", round(f1_score(y_test, y_pred), 2))
        print("\n")
        
wordEmbeddingOptimisationTFIDF()

/Users/shanecooke/miniforge3/envs/virt/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


TFIDF:

Max_features = 1000
-------------------
Accuracy:  0.946667
Precision:  0.96
Recall:  0.78
F1 Score:  0.86


Max_features = 2000
-------------------
Accuracy:  0.956667
Precision:  0.94
Recall:  0.83
F1 Score:  0.89


Max_features = 3000
-------------------
Accuracy:  0.937778
Precision:  0.97
Recall:  0.74
F1 Score:  0.84




In [5]:
print("Doc2Vec:\n")
params = ["Epochs = 10", "Epochs = 20", "Epochs = 40"]

def wordEmbeddingOptimisationDoc2Vec():
    for i in range(0, 3):
        X, y = df[df.columns[i+7]].tolist(), df.Hateful
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        
        classifier = LinearDiscriminantAnalysis(solver='eigen')
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        
        prec = precision_score(y_test, y_pred)
        rec = recall_score(y_test, y_pred)
        f1score = f1_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(params[i])
        print("-------------------")
        print("Accuracy: ", round(accuracy_score(y_test, y_pred), 6))
        print("Precision: ", round(precision_score(y_test, y_pred), 2))
        print("Recall: ", round(recall_score(y_test, y_pred),2))
        print("F1 Score: ", round(f1_score(y_test, y_pred), 2))
        print("\n")
        
wordEmbeddingOptimisationDoc2Vec()

Doc2Vec:

Epochs = 10
-------------------
Accuracy:  0.808889
Precision:  0.0
Recall:  0.0
F1 Score:  0.0


Epochs = 20
-------------------
Accuracy:  0.792222
Precision:  0.46
Recall:  0.06
F1 Score:  0.11


Epochs = 40
-------------------
Accuracy:  0.897778
Precision:  0.91
Recall:  0.55
F1 Score:  0.68




In [6]:
print("Hashing:\n")
params = ["N_features = 100", "N_features = 300", "N_features = 500"]

def wordEmbeddingOptimisationHashing():
    for i in range(0, 3):
        X, y = df[df.columns[i+10]].tolist(), df.Hateful
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        
        classifier = XGBClassifier(verbosity = 0)
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        
        prec = precision_score(y_test, y_pred)
        rec = recall_score(y_test, y_pred)
        f1score = f1_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(params[i])
        print("-------------------")
        print("Accuracy: ", round(accuracy_score(y_test, y_pred), 6))
        print("Precision: ", round(precision_score(y_test, y_pred), 2))
        print("Recall: ", round(recall_score(y_test, y_pred),2))
        print("F1 Score: ", round(f1_score(y_test, y_pred), 2))
        print("\n")
        
wordEmbeddingOptimisationHashing()

Hashing:

N_features = 100
-------------------
Accuracy:  0.86
Precision:  0.62
Recall:  0.53
F1 Score:  0.57


N_features = 300
-------------------
Accuracy:  0.895556
Precision:  0.81
Recall:  0.59
F1 Score:  0.68


N_features = 500
-------------------
Accuracy:  0.898889
Precision:  0.89
Recall:  0.6
F1 Score:  0.72


